In [1]:
import numpy as np
import pandas as pd
import io
import bson
import matplotlib.pyplot as plt
from skimage.data import imread
import multiprocessing as mp
import pickle
import tensorflow as tf

In [12]:
NCORE = 2

all_categories_array = np.array([])

#categories to int dictionary
categ_to_int = {}
int_to_categ = {}

#total number of items in the list
n_train = 7069896 #from kaggle page
n_test = 1768182 #from kaggle page
n_example = 100 #from kaggle page

all_categories_filename_format = 'allcategoriesdata_{0}.p'
train_data_batch_file_format = 'training_batches/{0}/train_{0}_{1}_{2}.jpeg'
test_data_batch_file_format = 'testing_batches/{0}/test_{0}_{1}_{2}.jpeg'

train_category_folder_path_format = 'training_batches/{0}'
test_category_folder_path_format = 'testing_batches/{0}'
test_category_folder_name_format = 'folder_{0}'

show_every = 10000

mini_batch = 1000

In [5]:
import time
import os.path

In [6]:
def load_categ_to_int_dicts(data_file_path):
    """
    restores categ_to_int and int_to_categ object dictionaries from saved state files if exist
    : data_file_path: actual data file path - to represent the mode (train or train example)
    """
    process_filename = data_file_path[data_file_path.rfind('/')+1:]
    filename_suffix = process_filename.replace('.bson','')
    categories_filename = all_categories_filename_format.format(filename_suffix)
    
    with open(categories_filename, 'rb') as f:
        
        global categ_to_int, int_to_categ
        
        categ_to_int, int_to_categ = pickle.load(f)

In [14]:
def create_one_hot_label(original_label, label_length, one_hot_labels):
    """
    creates one hot label for a given original label value. A sub function for multi core processing of one hot encode function
    : label_length: length of label to initialize the array
    : one_hot_labels: the array that contains all one hot label
    : return: void
    """
    one_hot_label = np.zeros(label_length, dtype='int16')
    one_hot_label[categ_to_int[original_label]] = 1
    one_hot_labels.append(one_hot_label)

def one_hot_encode(data_batch):
    """
    creates one hot encoded label for the given data batch using multi-core processing
    : data_batch: the sub-section of original final training data
    : return: array of one hot encoded label
    """
    one_hot_labels = list()
    label_length = len(categ_to_int)
    #print(data_batch)
    for i in range(len(data_batch)):
        original_label = data_batch[i][0] # 0 - category column
        create_one_hot_label(original_label, label_length, one_hot_labels)

    one_hot_labels = np.array(list(one_hot_labels))
    return one_hot_labels

In [11]:
#Load dictionaries - categ_to_int and int_to_categ from files to objects
load_categ_to_int_dicts('data/train.bson')

In [10]:
len(categ_to_int)

5270

In [13]:
def normalize(x):
    """
    Normalize a list of sample image data in the range of 0 to 1
    : x: List of image data.  The image shape is (180, 180, 3)
    : return: Numpy array of normalize data
    """
    xmax = 255 #image max value
    return x.astype(np.float)/float(xmax)


In [15]:
data_dir = 'training_batches/'
contents = os.listdir(data_dir)
classes = [each for each in contents if os.path.isdir(data_dir + each)]

In [29]:
def load_image(path):
    """
    loads the image from the given path, crops if it is not 180x180 and returns the image data
    : path: image file path
    : returns: resized image data
    """
    img = imread(path)
    img = img / 255.0
    if (img.shape[0] == 180) & (img.shape[1] == 180):
        return img
    else:
        short_edge = min(img.shape[:2])
        yy = int((img.shape[0] - short_edge) / 2)
        xx = int((img.shape[1] - short_edge) / 2)
        crop_img = img[yy: yy + short_edge, xx: xx + short_edge]
        # resize to 180, 180
        resized_img = skimage.transform.resize(crop_img, (180, 180), mode='constant')
        return resized_img

'''Test Method below'''
#load_image('training_batches/1000000237/train_1000000237_12600_0.jpeg')
print('Image Load function done')

Image Load function done


In [54]:
file_label_mapping = 'file_label_mapping.p'
file_path_format = '{0}/{1}/{2}'
label_folder_format = '{0}/{1}'

def fetch_filenames_labels_train(folder_path):
    """
    fetches all the filenames and their labels and dumps them into a pickle files
    : folder_path: path of the parent folder
    : returns: void
    """
    contents = os.listdir(folder_path)
    all_labels = [each for each in contents if os.path.isdir(label_folder_format.format(folder_path, each))]
    labels = list()
    inputs = list()
    for label_folder in all_labels:
        img_files = os.listdir(label_folder_format.format(folder_path, label_folder))
        inputs.extend([label_folder_format.format(folder_path, label_folder, each) for each in img_files])
        labels.extend([label_folder for each in img_files])
    pickle.dump((inputs, labels), open(file_label_mapping, 'wb'))

In [44]:
fetch_filenames_labels_train('training_batches')

In [56]:
def restore_filenames_labels_train(file_path):
    """
    loads the pickle file that has information of image file paths and their respective labels 
    : file_path: pickle file path
    : returns: inputs (file paths) and labels
    """
    if(os.path.exists(file_path)):
        with open(file_path, 'rb') as f:
            inputs, labels = pickle.load(f)
            return inputs, labels

In [46]:
inputs, labels = restore_filenames_labels_train(file_label_mapping)

In [51]:
from collections import Counter
ctr = Counter(labels)

In [57]:
inputs_array = np.array(inputs)
labels_array = np.array(labels)

In [63]:
from sklearn.model_selection import StratifiedShuffleSplit

def get_training_val_test_sets(inputs_array, labels_array):
    
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.4)
    train_idx, val_idx = next(splitter.split(inputs_array, labels_array))

    half_val_len = int(len(val_idx)/2)
    val_idx, test_idx = val_idx[:half_val_len], val_idx[half_val_len:]

    train_x, train_y = inputs_array[train_idx], labels_array[train_idx]
    val_x, val_y = inputs_array[val_idx], labels_array[val_idx]
    test_x, test_y = inputs_array[test_idx], labels_array[test_idx]
    
    result = [[train_x, train_y], [val_x, val_y], [test_x, test_y]]
    
    return result

In [67]:
#inspired by resnet50 (infact, trying to recreate resnet50)
def identity_block(input_tensor, kernel_size, filters, stage, block):
    """
    creates an identity block. Identity layer is a layer that has no conv layer at shortcut
    : input_tensor: input tensor
    : kernel_size: default 3, kernel size of the middle layer
    : filters: list of integers, filter sizes of three conv layers
    : stage: current stage, integer, used for creating names
    : block: current block, character, used for creating names
    """
    filter1, filter2, filter3 = filters
    bn_axis = 3
    
    conv_name = 'res_{0}_{1}_branch_'.format(str(stage), block)
    bn_name = 'bn_{0}_{1}_branch_'.format(str(stage), block)
    
    #kernel_size is fed, strides=(1,1) default, padding=valid default
    x = tf.layers.conv2d(input_tensor, filter1, (1,1), name=conv_name + '2a') 
    x = tf.layers.batch_normalization(x, axis=bn_axis, name=bn_name +'2a')
    x = tf.nn.relu(x)
    
    x = tf.layers.conv2d(x, filter2, kernel_size=kernel_size, padding='same', name= conv_name + '2b')
    x = tf.layers.batch_normalization(x, axis=bn_axis, name=bn_name +'2b')
    x = tf.nn.relu(x)
    
    x = tf.layers.conv2d(x, filter3, (1,1), name=conv_name + '2c')
    x = tf.layers.batch_normalization(x, axis=bn_axis, name=bn_name +'2c')
    
    x = tf.add(x, input_tensor) #short cut connection
    x = tf.nn.relu(x)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """
    creates an conv block. Conv block layer is a layer that has a conv layer at shortcut
    : input_tensor: input tensor
    : kernel_size: default 3, kernel size of the middle layer
    : filters: list of integers, filter sizes of three conv layers
    : stage: current stage, integer, used for creating names
    : block: current block, character, used for creating names
    : strides: strides that kernels take
    """
    filter1, filter2, filter3 = filters
    bn_axis = 3
    
    conv_name = 'res_{0}_{1}_branch_'.format(str(stage), block)
    bn_name = 'bn_{0}_{1}_branch_'.format(str(stage), block)
    
    x = tf.layers.conv2d(input_tensor, filter1, (1,1), strides=strides, name=conv_name + '2a') #applied strides
    x = tf.layers.batch_normalization(x, axis=bn_axis, name=bn_name +'2a')
    x = tf.nn.relu(x)
    
    x = tf.layers.conv2d(x, filter2, kernel_size, padding='same', name=conv_name + '2b')
    x = tf.layers.batch_normalization(x, axis=bn_axis, name=bn_name +'2b')
    x = tf.nn.relu(x)
    
    x = tf.layers.conv2d(x, filter3, (1,1), padding='same', name=conv_name + '2c')
    x = tf.layers.batch_normalization(x, axis=bn_axis, name=bn_name +'2c')
    
    shortcut = tf.layers.conv2d(input_tensor, filter3, (1,1), strides=strides, name=conv_name + '1')
    shortcut = tf.layers.batch_normalization(shortcut, axis=bn_axis, name=bn_name +'1')
    
    x = tf.add(x, shortcut) #short cut connection
    x = tf.nn.relu(x)
    return x

    

In [68]:
#inspired by resnet50 (infact, trying to recreate resnet50)
def build_model(image_shape, n_classes):
    
    bn_axis = 3
    #prepare input tensors
    x = tf.placeholder(tf.float32, shape=[None, *image_shape], name='x')
    y = tf.placeholder(tf.float32, shape=[None, n_classes], name='y')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    #prepare model
    x = tf.layers.conv2d(x, 64, (7,7), strides=(2,2), name='conv1')
    x = tf.layers.batch_normalization(x, axis=bn_axis, name='bn_conv1')
    x = tf.nn.relu(x)
    x = tf.layers.max_pooling2d(x, (3,3), strides=(2,2))
    
    x = conv_block(x, 3, [64,64,256], stage=2, block='a', strides=(1,1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
    
    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
    
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')
    
    x = conv_block(x, 3, [512,512,2048], stage=2, block='a')
    x = identity_block(x, 3, [512,512,2048], stage=2, block='b')
    x = identity_block(x, 3, [512,512,2048], stage=2, block='c')
    
    x = tf.layers.average_pooling2d(x, (7,7), name='avg_pool')
    
    #flattening
    image_size = x.get_shape()[1:].num_elements()
    x = tf.reshape(x, [-1, image_size])
    
    x = tf.layers.dense(x, n_classes, activation='softmax', name='fc5270')
    return x